In [ ]:
!apt-get install pandoc
!pip install pypandoc
!pip install python-docx
!pip install docx2txt

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
pandoc is already the newest version (2.9.2.1-3ubuntu2).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
#extracting folders id from shared memory

from google.colab import auth
from googleapiclient.discovery import build
import pandas as pd

# Authenticate
auth.authenticate_user()
drive_service = build('drive', 'v3')

folder_id = "1e0eA-AIsz_BSwVHOppJMXECX42hBfG4J"

def list_all_files_in_folder_recursive(parent_id, parent_path=""):
    """Recursively list all files in a folder and subfolders"""
    all_files = []

    query = f"'{parent_id}' in parents and trashed=false"
    page_token = None

    while True:
        response = drive_service.files().list(
            q=query,
            spaces='drive',
            fields='nextPageToken, files(id, name, mimeType)',
            pageToken=page_token
        ).execute()

        for item in response.get('files', []):
            item_path = f"{parent_path}/{item['name']}" if parent_path else item['name']

            if item['mimeType'] == 'application/vnd.google-apps.folder':
                # Recurse into subfolder
                all_files.extend(list_all_files_in_folder_recursive(item['id'], item_path))
            else:
                all_files.append({
                    "file_name": item['name'],
                    "file_path": item_path,
                    "file_id": item['id'],
                    "file_url": f"https://drive.google.com/file/d/{item['id']}/view?usp=sharing"
                })

        page_token = response.get('nextPageToken', None)
        if page_token is None:
            break

    return all_files

# Run the recursive function
files_list = list_all_files_in_folder_recursive(folder_id)

# Convert to DataFrame
df_files = pd.DataFrame(files_list)
##df_files.head()

##getting local id of the file
import os
from googleapiclient.http import MediaIoBaseDownload
import io

def get_chapter_file(chapter_num: int, year: str, download_dir="/content") -> str:
    """
    Download a chapter file for a given year from Google Drive and return its local path.

    Args:
        chapter_num (int): The chapter number (e.g., 1, 2, ..., 14).
        year (str): The year folder name.
        download_dir (str): Local folder to save the file. Defaults to /content in Colab.

    Returns:
        str: Local path to the downloaded file, or None if not found.
    """
    # Normalize chapter filename (01, 02, ...)
    chapter_str = f"{chapter_num:02}"

    # Find matching file in df_files
    match = df_files[
        (df_files["file_name"].str.contains(chapter_str)) &
        (df_files["file_path"].str.contains(f"{year}/"))
    ]

    if match.empty:
        print(f"❌ No file found for chapter {chapter_str} in year {year}")
        return None

    file_id = match.iloc[0]["file_id"]
    file_name = match.iloc[0]["file_name"]
    local_path = os.path.join(download_dir, year, file_name)

    # Ensure year directory exists locally
    os.makedirs(os.path.dirname(local_path), exist_ok=True)

    # Download the file from Google Drive
    request = drive_service.files().get_media(fileId=file_id)
    fh = io.FileIO(local_path, "wb")
    downloader = MediaIoBaseDownload(fh, request)

    done = False
    while not done:
        status, done = downloader.next_chunk()
        if status:
            print(f"⬇️ Download {int(status.progress() * 100)}%.")

    print(f"✅ Saved to {local_path}")
    return local_path

from docx import Document

# After downloading the file
# sanity check
path = get_chapter_file(14, "2015")
print("Local file:", path)

if path and path.endswith(".docx"):
    doc = Document(path)
    full_text = "\n".join([p.text for p in doc.paragraphs if p.text.strip() != ""])

    # Split into sentences (naive split by period/full stop)
    sentences = full_text.replace("\n", " ").split(".")

    # Print first few sentences
    preview_count = 3
    preview_sentences = [s.strip() for s in sentences if s.strip()][:preview_count]
    print("\nPreview of content:")
    for i, s in enumerate(preview_sentences, 1):
        print(f"{i}. {s}.")


#a code that downloads the files to a directory called content/reports
import os
import io
from googleapiclient.http import MediaIoBaseDownload

def download_all_chapters(download_dir="/content/reports", years=range(2001, 2025), chapters=range(1, 16)):
    """
    Downloads all chapters for all years to local environment.

    Returns:
        dict: {year: {chapter_number: local_path}}
    """
    all_paths = {}

    for year in years:
        year_str = str(year)
        all_paths[year_str] = {}

        for chapter in chapters:
            chapter_str = f"{chapter:02}"

            # Find matching file in df_files
            match = df_files[
                (df_files["file_name"].str.contains(chapter_str)) &
                (df_files["file_path"].str.contains(f"{year_str}/"))
            ]

            if match.empty:
                print(f"⚠️ Chapter {chapter_str} not found for year {year_str}")
                continue

            file_id = match.iloc[0]["file_id"]
            file_name = match.iloc[0]["file_name"]
            local_path = os.path.join(download_dir, year_str, f"{chapter_str}_{file_name}")

            # Ensure folder exists
            os.makedirs(os.path.dirname(local_path), exist_ok=True)

            # Download file
            request = drive_service.files().get_media(fileId=file_id)
            fh = io.FileIO(local_path, "wb")
            downloader = MediaIoBaseDownload(fh, request)

            done = False
            while not done:
                status, done = downloader.next_chunk()
                if status:
                    print(f"⬇️ Download {int(status.progress() * 100)}% for {file_name}")

            print(f"✅ Saved {file_name} to {local_path}")
            all_paths[year_str][chapter] = local_path

    return all_paths
download_all_chapters()

⬇️ Download 100%.
✅ Saved to /content/2015/14.docx
Local file: /content/2015/14.docx

Preview of content:
1. 14 ילדים במצבי סיכון ומצוקה מבוא פרק זה, המוקדש לילדים במצבי סיכון ומצוקה, הוא אחד הפרקים המורכבים בשנתון.
2. ההגדרה מיהו ילד בסיכון או במצוקה תלויה בסביבה, בחברה, בתקופה ובמגדיר.
3. לא רק זאת, קשה לאמוד את היקף התופעות הקשורות לילדים במצבי סיכון ומצוקה, שכן לפי טבען לא תמיד הן חשופות או מדווחות, והמקרים הידועים אינם אלא קצה הקרחון.
⬇️ Download 100% for 01.docx
✅ Saved 01.docx to /content/reports/2001/01_01.docx
⬇️ Download 100% for 02.docx
✅ Saved 02.docx to /content/reports/2001/02_02.docx
⬇️ Download 100% for 03.docx
✅ Saved 03.docx to /content/reports/2001/03_03.docx
⬇️ Download 100% for 04.docx
✅ Saved 04.docx to /content/reports/2001/04_04.docx
⬇️ Download 100% for 05.docx
✅ Saved 05.docx to /content/reports/2001/05_05.docx
⬇️ Download 100% for 06.docx
✅ Saved 06.docx to /content/reports/2001/06_06.docx
⬇️ Download 100% for 07.docx
✅ Saved 07.docx to /content/reports/2001/

{'2001': {1: '/content/reports/2001/01_01.docx',
  2: '/content/reports/2001/02_02.docx',
  3: '/content/reports/2001/03_03.docx',
  4: '/content/reports/2001/04_04.docx',
  5: '/content/reports/2001/05_05.docx',
  6: '/content/reports/2001/06_06.docx',
  7: '/content/reports/2001/07_07.docx',
  8: '/content/reports/2001/08_08.docx',
  9: '/content/reports/2001/09_09.docx',
  10: '/content/reports/2001/10_10.docx',
  11: '/content/reports/2001/11_11.docx',
  12: '/content/reports/2001/12_12.docx',
  13: '/content/reports/2001/13_13.docx',
  14: '/content/reports/2001/14_14.docx',
  15: '/content/reports/2001/15_15.docx'},
 '2002': {1: '/content/reports/2002/01_01.docx',
  2: '/content/reports/2002/02_02.docx',
  3: '/content/reports/2002/03_03.docx',
  4: '/content/reports/2002/04_04.docx',
  5: '/content/reports/2002/05_05.docx',
  6: '/content/reports/2002/06_06.docx',
  7: '/content/reports/2002/07_07.docx',
  8: '/content/reports/2002/08_08.docx',
  9: '/content/reports/2002/09_09.

In [ ]:
from google.colab import auth
from googleapiclient.discovery import build
import pandas as pd
import os
import io
from googleapiclient.http import MediaIoBaseDownload
import logging

# Set up logging
logger = logging.getLogger('__main__')
logger.setLevel(logging.INFO)


class GoogleDriveManager:
    """
    Manages Google Drive operations including listing, filtering, downloading, and uploading files.
    """

    def __init__(self, folder_id):
        """
        Initialize the GoogleDriveManager with authentication and folder ID.

        Args:
            folder_id: The Google Drive folder ID to work with
        """
        self.folder_id = folder_id
        self.drive_service = None
        self.files_df = None  # Cache for file listings

        # Authenticate and build service
        self._authenticate()

    def _authenticate(self):
        """Authenticate with Google Drive and build the service object."""
        try:
            auth.authenticate_user()
            self.drive_service = build('drive', 'v3')
            logger.info("✅ Successfully authenticated with Google Drive")
        except Exception as e:
            logger.error(f"❌ Authentication failed: {e}")
            raise

    def list_all_files(self, force_refresh=False):
        """
        Recursively list all files in the folder and subfolders.

        Args:
            force_refresh: If True, force a new listing even if cached data exists

        Returns:
            pd.DataFrame: DataFrame with columns [file_name, file_path, file_id, file_url]
        """
        if self.files_df is not None and not force_refresh:
            logger.info("📋 Using cached file list")
            return self.files_df

        logger.info("🔍 Listing all files in folder...")
        all_files = self._list_files_recursive(self.folder_id)

        # Convert to DataFrame
        if all_files:
            self.files_df = pd.DataFrame(all_files)

            # Deduplicate by folder+name (file_path already encodes folder)
            self.files_df = self.files_df.drop_duplicates(
                subset=["file_path", "file_name"], keep="first"
            )

            logger.info(f"✅ Found {len(self.files_df)} unique files")
        else:
            self.files_df = pd.DataFrame(columns=['file_name', 'file_path', 'file_id', 'file_url'])
            logger.info("📁 No files found in folder")

        return self.files_df

    def _list_files_recursive(self, parent_id, parent_path=""):
        """
        Recursively list files in a folder.

        Args:
            parent_id: Google Drive folder ID
            parent_path: Path string for tracking folder hierarchy

        Returns:
            list: List of file dictionaries
        """
        all_files = []
        query = f"'{parent_id}' in parents and trashed=false"
        page_token = None

        while True:
            try:
                response = self.drive_service.files().list(
                    q=query,
                    spaces='drive',
                    fields='nextPageToken, files(id, name, mimeType)',
                    pageToken=page_token
                ).execute()

                for item in response.get('files', []):
                    item_path = f"{parent_path}/{item['name']}" if parent_path else item['name']

                    if item['mimeType'] == 'application/vnd.google-apps.folder':
                        # Recurse into subfolder
                        all_files.extend(self._list_files_recursive(item['id'], item_path))
                    else:
                        all_files.append({
                            "file_name": item['name'],
                            "file_path": item_path,
                            "file_id": item['id'],
                            "file_url": f"https://drive.google.com/file/d/{item['id']}/view?usp=sharing"
                        })

                page_token = response.get('nextPageToken', None)
                if page_token is None:
                    break

            except Exception as e:
                logger.error(f"❌ Error listing files in {parent_path}: {e}")
                break

        return all_files

    def filter_files(self, df=None, years=None, chapters=None):
        """
        Filter files based on specified years and chapters using exact matching.

        Args:
            df: DataFrame to filter (if None, uses cached files_df)
            years: List of years to include (e.g., [2021, 2022, 2023])
            chapters: List of chapter numbers to include (e.g., [1, 2, 5, 10])

        Returns:
            pd.DataFrame: Filtered DataFrame containing only requested files
        """
        # Use provided df or cached one
        if df is None:
            if self.files_df is None:
                logger.warning("⚠️ No files listed yet. Running list_all_files() first.")
                self.list_all_files()
            df = self.files_df.copy()
        else:
            df = df.copy()

        if df.empty:
            logger.warning("⚠️ No files to filter")
            return df

        # Apply year filter
        if years is not None:
            year_strings = [str(year) for year in years]
            # Exact match: year must be a folder in the path
            year_mask = df['file_path'].apply(
                lambda path: any(f"/{year}/" in f"/{path}" or path.startswith(f"{year}/")
                               for year in year_strings)
            )
            df = df[year_mask]
            logger.info(f"📅 Filtered for years: {years} - {len(df)} files")

        # Apply chapter filter
        if chapters is not None:
            # Exact match for filename pattern: 01.docx, 02.docx, etc.
            chapter_filenames = [f"{ch:02d}.docx" for ch in chapters]
            chapter_mask = df['file_name'].apply(
                lambda name: name in chapter_filenames
            )
            df = df[chapter_mask]
            logger.info(f"📖 Filtered for chapters: {chapters} - {len(df)} files")

        return df

    def download_files(self, filtered_df, download_dir="/content/reports"):
        """
        Download files from a filtered DataFrame.

        Args:
            filtered_df: DataFrame containing files to download
            download_dir: Base directory for downloads

        Returns:
            dict: Dictionary mapping file paths to local paths
        """
        if filtered_df is None or filtered_df.empty:
            logger.warning("⚠️ No files to download")
            return {}

        downloaded_files = {}
        total_files = len(filtered_df)

        logger.info(f"📥 Starting download of {total_files} files...")

        for idx, row in filtered_df.iterrows():
            file_id = row['file_id']
            file_name = row['file_name']
            file_path = row['file_path']

            # Extract year from path (assuming structure: year/filename)
            path_parts = file_path.split('/')
            if len(path_parts) >= 2:
                year = path_parts[0]
                local_path = os.path.join(download_dir, year, file_name)
            else:
                local_path = os.path.join(download_dir, file_name)

            # Ensure directory exists
            os.makedirs(os.path.dirname(local_path), exist_ok=True)

            try:
                # Download file
                request = self.drive_service.files().get_media(fileId=file_id)
                fh = io.FileIO(local_path, "wb")
                downloader = MediaIoBaseDownload(fh, request)

                done = False
                while not done:
                    status, done = downloader.next_chunk()
                    if status:
                        progress = int(status.progress() * 100)
                        print(f"⬇️  Downloading {file_name}: {progress}%", end='\r')

                logger.info(f"✅ Downloaded {file_name} to {local_path}")
                downloaded_files[file_path] = local_path

            except Exception as e:
                logger.warning(f"⚠️ Failed to download {file_name}: {e}")
                continue

        logger.info(f"✅ Download complete: {len(downloaded_files)}/{total_files} files")
        return downloaded_files

    def download_selective(self, years=None, chapters=None, download_dir="/content/reports"):
        """
        Convenience method to list, filter, and download files in one operation.

        Args:
            years: List of years to download (e.g., [2021, 2022, 2023])
            chapters: List of chapter numbers to download (e.g., [1, 2, 5, 10])
            download_dir: Base directory for downloads

        Returns:
            dict: Dictionary mapping file paths to local paths

        Example:
            # Download chapters 1-5 for years 2021-2023
            manager.download_selective(
                years=range(2021, 2024),
                chapters=range(1, 6),
                download_dir="/content/reports"
            )
        """
        # Step 1: List all files
        logger.info("🚀 Starting selective download workflow...")
        all_files = self.list_all_files()

        # Step 2: Filter files
        filtered_files = self.filter_files(all_files, years=years, chapters=chapters)

        if filtered_files is None or filtered_files.empty:
            logger.warning("⚠️ No files match the specified criteria")
            return {}

        logger.info(f"📊 Found {len(filtered_files)} files matching criteria")

        # Step 3: Download filtered files
        downloaded = self.download_files(filtered_files, download_dir)

        return downloaded

    def get_summary(self, df=None):
        """
        Get summary statistics about the files.

        Args:
            df: DataFrame to summarize (if None, uses cached files_df)

        Returns:
            dict: Summary statistics
        """
        if df is None:
            if self.files_df is None:
                logger.warning("⚠️ No files listed yet. Running list_all_files() first.")
                self.list_all_files()
            df = self.files_df

        if df is None or df.empty:
            return {"total_files": 0, "years": [], "chapters": []}

        # Extract years from paths
        years = df['file_path'].apply(lambda x: x.split('/')[0] if '/' in x else None)
        years = sorted(years.dropna().unique())

        # Extract chapters from filenames (assuming pattern: 01.docx, 02.docx)
        chapters = df['file_name'].apply(
            lambda x: int(x[:2]) if x[:2].isdigit() and x.endswith('.docx') else None
        )
        chapters = sorted(chapters.dropna().unique())

        summary = {
            "total_files": len(df),
            "years": years,
            "year_count": len(years),
            "chapters": chapters,
            "chapter_count": len(chapters),
            "file_types": df['file_name'].apply(lambda x: x.split('.')[-1]).value_counts().to_dict()
        }

        return summary

    def preview_files(self, df=None, n=10):
        """
        Preview first n files from the DataFrame.

        Args:
            df: DataFrame to preview (if None, uses cached files_df)
            n: Number of files to preview
        """
        if df is None:
            if self.files_df is None:
                logger.warning("⚠️ No files listed yet. Running list_all_files() first.")
                self.list_all_files()
            df = self.files_df

        if df is None or df.empty:
            logger.info("No files to preview")
            return

        preview = df.head(n)[['file_name', 'file_path']]
        logger.info(f"\n📋 Preview of first {min(n, len(df))} files:")
        for idx, row in preview.iterrows():
            logger.info(f"  {row['file_path']}")

    def check_missing_files(self, years, chapters):
        """
        Check which year/chapter combinations are missing.

        Args:
            years: List of years to check
            chapters: List of chapter numbers to check

        Returns:
            list: List of missing (year, chapter) tuples
        """
        if self.files_df is None:
            self.list_all_files()

        missing = []

        for year in years:
            for chapter in chapters:
                # Check if this combination exists
                filtered = self.filter_files(
                    self.files_df,
                    years=[year],
                    chapters=[chapter]
                )

                if filtered is None or filtered.empty:
                    missing.append((year, chapter))
                    logger.warning(f"⚠️ Missing: Year {year}, Chapter {chapter:02d}")

        if missing:
            logger.info(f"📊 Total missing files: {len(missing)}")
        else:
            logger.info("✅ All requested files are present")

        return missing

In [ ]:
# Initialize
folder_id = "1e0eA-AIsz_BSwVHOppJMXECX42hBfG4J"
manager = GoogleDriveManager(folder_id)

# Download specific years and chapters
downloaded = manager.download_selective(
    years=range(2001, 2025),
    chapters=range(1, 16)
)

print(f"Downloaded {len(downloaded)} files")

INFO:__main__:✅ Successfully authenticated with Google Drive
INFO:__main__:🚀 Starting selective download workflow...
INFO:__main__:🔍 Listing all files in folder...
INFO:__main__:✅ Found 882 unique files
INFO:__main__:📅 Filtered for years: range(2001, 2025) - 882 files
INFO:__main__:📖 Filtered for chapters: range(1, 16) - 361 files
INFO:__main__:📊 Found 361 files matching criteria
INFO:__main__:📥 Starting download of 361 files...
INFO:__main__:✅ Downloaded 04.docx to /content/reports/2016/04.docx


INFO:__main__:✅ Downloaded 11.docx to /content/reports/2016/11.docx


INFO:__main__:✅ Downloaded 14.docx to /content/reports/2016/14.docx


INFO:__main__:✅ Downloaded 02.docx to /content/reports/2016/02.docx


INFO:__main__:✅ Downloaded 09.docx to /content/reports/2016/09.docx


...

INFO:__main__:✅ Downloaded 08.docx to /content/reports/2014/08.docx


INFO:__main__:✅ Downloaded 10.docx to /content/reports/2014/10.docx


INFO:__main__:✅ Downloaded 03.docx to /content/reports/2014/03.docx


INFO:__main__:✅ Downloaded 06.docx to /content/reports/2014/06.docx
INFO:__main__:✅ Download complete: 361/361 files


Downloaded 361 files


# Old code for extracting tables


In [ ]:
import os
import json
import pandas as pd
from docx import Document

def extract_tables_from_reports(base_dir="/content/reports", out_dir="/content/tables"):
    os.makedirs(out_dir, exist_ok=True)
    summary = {}
    colnames_map = {}

    for year in range(2001, 2025):  # adjust later
        print(year)
        year_path = os.path.join(base_dir, str(year))
        if not os.path.isdir(year_path):
            continue

        for fname in os.listdir(year_path):
            if not fname.endswith(".docx"):
                continue

            chapter = fname.split("_")[0]
            fpath = os.path.join(year_path, fname)

            try:
                doc = Document(fpath)
            except Exception as e:
                print(f"skip {fpath}: {e}")
                continue

            serial = 1
            # find all tables with names containing "לוח"
            for i, table in enumerate(doc.tables):
                table_name = ""  # default

                # check the first row cells for "לוח"
                if len(table.rows) > 0:
                    for cell in table.rows[0].cells:
                        if "לוח" in cell.text and "תרשים" not in cell.text:  # extra check
                            table_name = cell.text.strip()
                            break

                    if not table_name:  # skip if not a valid table
                        continue

                    # convert table to dataframe
                    data = [[cell.text.strip() for cell in row.cells] for row in table.rows]
                    df = pd.DataFrame(data)

                    identifier = f"{serial}_{chapter}_{year}"

                    # RWMOVING THIS
                    # record mapping for JSON
                    # if len(df) > 0:
                    #    summary[identifier] = " ".join(df.iloc[0].astype(str).tolist())
                    # else:
                    #    continue

                    #if len(df) > 1:
                    #    colnames_map[identifier] = df.iloc[1].astype(str).tolist()
                    #else:
                    #    colnames_map[identifier] = []

                    # ADD THIS
                    # record mapping for JSON
                    if len(df) > 0:
                        # Deduplicate consecutive repeated text in header
                        header_cells = df.iloc[0].astype(str).tolist()
                        unique_header = []
                        for cell in header_cells:
                            if not unique_header or cell != unique_header[-1]:
                                unique_header.append(cell)
                        summary[identifier] = " ".join(unique_header)
                    else:
                        continue

                    # Combine rows [1] and [2] for column names
                    if len(df) > 2:
                        row1 = df.iloc[1].astype(str).tolist()
                        row2 = df.iloc[2].astype(str).tolist()
                        colnames_map[identifier] = [f"{r1} {r2}".strip() for r1, r2 in zip(row1, row2)]
                    elif len(df) > 1:
                        colnames_map[identifier] = df.iloc[1].astype(str).tolist()
                    else:
                        colnames_map[identifier] = []

                    #FROM HERE ON IT'S AS IT WAS BEFORE (SHAUL 08.09)
                    # save CSV
                    save_dir = os.path.join(out_dir, str(year), chapter)
                    os.makedirs(save_dir, exist_ok=True)
                    save_path = os.path.join(save_dir, f"{identifier}.csv")
                    df.to_csv(save_path, index=False, encoding="utf-8-sig")

                    serial += 1

    # write summary JSON
    with open(os.path.join(out_dir, "tables_summary.json"), "w", encoding="utf-8") as f:
        json.dump(summary, f, ensure_ascii=False, indent=2)

    with open(os.path.join(out_dir, "tables_columns.json"), "w", encoding="utf-8") as f:
        json.dump(colnames_map, f, ensure_ascii=False, indent=2)


extract_tables_from_reports()


2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024


# New code for extracting tables

In [ ]:
import os
import json
import pandas as pd
from docx import Document

# Configuration Constants
YEAR_RANGE = (2001, 2025)  # Years to process
VALID_EXTENSION = ".docx"
TABLE_MARKER = "לוח"  # Hebrew for "table"
EXCLUDE_MARKER = "תרשים"  # Hebrew for "diagram" - exclude these
ENCODING = "utf-8-sig"
SUMMARY_FILE = "tables_summary.json"
COLUMNS_FILE = "tables_columns.json"

def is_valid_table(table):
    """
    Check if a table is valid (contains Hebrew table marker in first row).

    Args:
        table: A docx table object

    Returns:
        tuple: (is_valid: bool, table_name: str)
    """
    if len(table.rows) == 0:
        return False, ""

    # Check first row cells for table marker
    for cell in table.rows[0].cells:
        cell_text = cell.text
        if TABLE_MARKER in cell_text and EXCLUDE_MARKER not in cell_text:
            return True, cell_text.strip()

    return False, ""


def extract_table_data(table):
    """
    Extract data from a docx table and convert to DataFrame.

    Args:
        table: A docx table object

    Returns:
        pd.DataFrame: Table data as a DataFrame
    """
    data = [[cell.text.strip() for cell in row.cells] for row in table.rows]
    return pd.DataFrame(data)


def save_table_data(df, identifier, year, chapter, out_dir):
    """
    Save DataFrame as CSV file in the appropriate directory structure.

    Args:
        df: pandas DataFrame to save
        identifier: Unique identifier for the table
        year: Year of the document
        chapter: Chapter identifier
        out_dir: Base output directory

    Returns:
        str: Path where the file was saved
    """
    save_dir = os.path.join(out_dir, str(year), chapter)
    os.makedirs(save_dir, exist_ok=True)

    save_path = os.path.join(save_dir, f"{identifier}.csv")
    df.to_csv(save_path, index=False, encoding=ENCODING)

    return save_path


def process_document(fpath, year, chapter, out_dir):
    """
    Process a single Word document and extract all valid tables.

    Args:
        fpath: Full path to the document
        year: Year of the document
        chapter: Chapter identifier from filename
        out_dir: Base output directory

    Returns:
        tuple: (summary_dict, colnames_dict) containing extracted data
    """
    summary = {}
    colnames_map = {}

    try:
        doc = Document(fpath)
    except Exception as e:
        print(f"skip {fpath}: {e}")
        return summary, colnames_map

    serial = 1

    for table in doc.tables:
        # Validate table
        is_valid, table_name = is_valid_table(table)
        if not is_valid:
            continue

        # Extract data
        df = extract_table_data(table)

        # Skip empty tables
        if len(df) == 0:
            continue

        # Create identifier
        identifier = f"{serial}_{chapter}_{year}"

        # Record table summary (first row)
        summary[identifier] = " ".join(df.iloc[0].astype(str).tolist())

        # Record column names (second row if exists)
        if len(df) > 1:
            colnames_map[identifier] = df.iloc[1].astype(str).tolist() ## add 2 and 3
        else:
            colnames_map[identifier] = []

        # Save to CSV
        save_table_data(df, identifier, year, chapter, out_dir)

        serial += 1
      # fix table name to not have duplicates due to multiple cells
    return summary, colnames_map


def save_metadata(summary, colnames_map, out_dir):
    """
    Save summary and column metadata to JSON files.

    Args:
        summary: Dictionary of table summaries
        colnames_map: Dictionary of column names
        out_dir: Output directory
    """
    with open(os.path.join(out_dir, SUMMARY_FILE), "w", encoding="utf-8") as f:
        json.dump(summary, f, ensure_ascii=False, indent=2)

    with open(os.path.join(out_dir, COLUMNS_FILE), "w", encoding="utf-8") as f:
        json.dump(colnames_map, f, ensure_ascii=False, indent=2)


def extract_tables_from_reports(base_dir="/content/reports", out_dir="/content/tables"):
    """
    Extract tables from Word documents organized by year and chapter.

    Processes all .docx files in year subdirectories, extracts tables containing
    Hebrew table markers, and saves them as CSV files with metadata JSON files.

    Args:
        base_dir: Root directory containing year subdirectories with reports
        out_dir: Output directory for extracted tables and metadata
    """
    os.makedirs(out_dir, exist_ok=True)

    # Initialize collectors for all documents
    all_summaries = {}
    all_colnames = {}

    # Process each year
    for year in range(*YEAR_RANGE):
        print(year)
        year_path = os.path.join(base_dir, str(year))

        if not os.path.isdir(year_path):
            continue

        # Process each document in the year directory
        for fname in os.listdir(year_path):
            if not fname.endswith(VALID_EXTENSION):
                continue

            # Extract chapter from filename
            chapter = fname.split("_")[0]
            fpath = os.path.join(year_path, fname)

            # Process the document
            doc_summary, doc_colnames = process_document(fpath, year, chapter, out_dir)

            # Merge results
            all_summaries.update(doc_summary)
            all_colnames.update(doc_colnames)

    # Save consolidated metadata
    save_metadata(all_summaries, all_colnames, out_dir)